In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torchvision.models as models
import torchvision
import glob
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector"
images_path = f"{folder_path}/Dataset/images"
annots_path = f"{folder_path}/Dataset/annotations"
test_path = f"{folder_path}/Test"

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

In [ ]:
num_classes = 3

model = models.detection.fasterrcnn_resnet50_fpn(weights='COCO_V1')
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
modelpath = "model"

model.load_state_dict(torch.load(f"{folder_path}/{modelpath}/model_state_dict.pt"))

<All keys matched successfully>

In [ ]:
transforms = torchvision.transforms.Compose([
      torchvision.transforms.ToTensor()
])

In [ ]:
class MaskTestDataset(torch.utils.data.Dataset):
    def __init__(self, transforms=None):
        self.transforms = transforms
        self.images = glob.glob(f"{test_path}/*.jpg")

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)

        return image, img_path
        

    def __len__(self):
        return len(self.images)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

test_dataset = MaskTestDataset(transforms=transforms)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    drop_last=False,
    collate_fn=collate_fn
)

In [76]:
model.eval()
model.to(device)
test_path = f"{folder_path}/Test"

output_list = []
image_list = []
for images, image_path in test_dataloader:
    images = list(image.to(device) for image in images)
    output = model(images)
    output_list.append(output)
    image_list.append(image_path)

# print(output_list)
# print(image_list)

In [73]:
def decode_category(label):
    if label == 0:
        return "without mask"
    elif label == 1:
        return "with mask"
    return "incorrect mask"

In [74]:
threshold = 0.9
white = (255, 255, 255)
font = cv2.FONT_HERSHEY_DUPLEX 
yellow = [0, 255, 255]

for output, image_path in zip(output_list, image_list):
    item = output[0]
    img_path = image_path[0]

    boxes = item['boxes'].cpu().detach().numpy()
    labels = item['labels'].cpu().detach().numpy()
    scores = item['scores'].cpu().detach().numpy()

    boxes = boxes[scores >= threshold]
    labels = labels[scores >= threshold]

    img = cv2.imread(img_path)
    col, row, _ = img.shape
    base = row / 3354
    resize_4 = max(1, int(4*base))
    resize_5 = max(1, int(5*base))
    resize_20 = max(1, int(20*base))

    for box, label in zip(boxes, labels):
        xmin_pred = int(box[0])
        ymin_pred = int(box[1])
        xmax_pred = int(box[2])
        ymax_pred = int(box[3])
        img[ymin_pred-resize_5:ymin_pred, xmin_pred:xmax_pred, :] = yellow
        img[ymax_pred:ymax_pred+resize_5, xmin_pred:xmax_pred, :] = yellow
        img[ymin_pred:ymax_pred, xmin_pred-resize_5:xmin_pred, :] = yellow
        img[ymin_pred:ymax_pred, xmax_pred:xmax_pred+resize_5, :] = yellow
        
        x_text = xmin_pred
        y_text = ymin_pred - resize_20
        category = decode_category(label)
        img = cv2.putText(img, category, (x_text, y_text), font, resize_4, white, 2, cv2.LINE_AA)
    new_path = f"{img_path[:-4]}_detect.jpg"
    print(new_path)
    cv2.imwrite(new_path, img)

/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/kobby-mendez-VGYcVZguFzs-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/engin-akyurt-WBM97UGM0QA-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/engin-akyurt-Y-K06KGg7ak-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/brian-asare-z7lTC8cFKKs-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/karina-carvalho-fKTKVrNqXQQ-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/imansyah-muhamad-putera-n4KewLKFOZw-unsplash_detect.jpg
/content/drive/MyDrive/Colab Notebooks/2022/Face Mask Detector/Test/christopher-campbell-rDEOVtE7vOs-unsplash_detect.jpg
